# Phase 3: Object Detection Performance Evaluation
Training and comparing YOLOv8 and RT-DETR on raw vs enhanced outdoor images.

**Metrics**: mAP@0.5, mAP@0.5:0.95, Precision, Recall, FPS
**Goal**: Determine best detection model and impact of image enhancement

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
PROJECT_DIR = '/content/drive/MyDrive/computer_vision'
DATASETS_DIR = f'{PROJECT_DIR}/datasets'
RESULTS_DIR = f'{PROJECT_DIR}/results/phase3'
os.makedirs(RESULTS_DIR, exist_ok=True)

%cd /content
!rm -rf computer_vision_expirement
!git clone https://github.com/Ib-Programmer/computer_vision_expirement.git
%cd computer_vision_expirement
!pip install -q ultralytics albumentations

## 3.1 Prepare Dataset in YOLO Format

In [ ]:
# Create dataset.yaml for YOLO training
# Using BDD100K for object detection (has detection labels)
import yaml

dataset_config = {
    'path': f'{DATASETS_DIR}/bdd100k_processed',
    'train': 'train',
    'val': 'val',
    'test': 'test',
    'names': {
        0: 'pedestrian',
        1: 'rider',
        2: 'car',
        3: 'truck',
        4: 'bus',
        5: 'train',
        6: 'motorcycle',
        7: 'bicycle',
        8: 'traffic light',
        9: 'traffic sign'
    }
}

with open('dataset.yaml', 'w') as f:
    yaml.dump(dataset_config, f)

print("Dataset config created: dataset.yaml")
print(f"Classes: {list(dataset_config['names'].values())}")

## 3.3 Train YOLOv8 (with outdoor augmentations enabled)
The outdoor augmentations patch above is now active. Every training image has a chance of being randomly degraded with fog, rain, low-light, motion blur, or combined effects — different every epoch.

from ultralytics import YOLO

# Train YOLOv8n with outdoor augmentations (patched above)
print("=" * 60)
print("Training YOLOv8n WITH outdoor augmentations (fog/rain/blur/dark/combined)")
print("=" * 60)

model_aug = YOLO('yolov8n.pt')
results_aug = model_aug.train(
    data='dataset.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    device=0,
    project=RESULTS_DIR,
    name='yolov8n_outdoor_aug',
    patience=10,
    save=True,
    plots=True
)

In [ ]:
# Train YOLOv8n WITHOUT outdoor augmentations (baseline comparison)
# Restore default albumentations to get a fair baseline
from ultralytics.data.augment import Albumentations as _Alb
import importlib, ultralytics.data.augment
importlib.reload(ultralytics.data.augment)
from ultralytics.data.augment import Albumentations

print("=" * 60)
print("Training YOLOv8n WITHOUT outdoor augmentations (baseline)")
print("=" * 60)

model_raw = YOLO('yolov8n.pt')
results_raw = model_raw.train(
    data='dataset.yaml',
    epochs=50,
    imgsz=640,
    batch=16,
    device=0,
    project=RESULTS_DIR,
    name='yolov8n_baseline',
    patience=10,
    save=True,
    plots=True
)

# Re-apply outdoor augmentation patch for RT-DETR
Albumentations.__init__ = _outdoor_init
print("\nOutdoor augmentations re-enabled for RT-DETR training")

## 3.4 Train RT-DETR (with outdoor augmentations)

# Train RT-DETR (transformer-based detector) with outdoor augmentations
print("=" * 60)
print("Training RT-DETR WITH outdoor augmentations")
print("=" * 60)

model_rtdetr = YOLO('rtdetr-l.pt')
results_rtdetr = model_rtdetr.train(
    data='dataset.yaml',
    epochs=50,
    imgsz=640,
    batch=8,  # smaller batch for transformer model
    device=0,
    project=RESULTS_DIR,
    name='rtdetr_outdoor_aug',
    patience=10,
    save=True,
    plots=True
)

## 3.5 Evaluate All Models
Compare baseline (no outdoor augmentation) vs outdoor-augmented training for both architectures.

import pandas as pd

# Validate each model
models = {
    'YOLOv8n_baseline': f'{RESULTS_DIR}/yolov8n_baseline/weights/best.pt',
    'YOLOv8n_outdoor_aug': f'{RESULTS_DIR}/yolov8n_outdoor_aug/weights/best.pt',
    'RT-DETR_outdoor_aug': f'{RESULTS_DIR}/rtdetr_outdoor_aug/weights/best.pt',
}

results_table = []

for name, weights_path in models.items():
    if os.path.exists(weights_path):
        print(f"\nValidating: {name}")
        model = YOLO(weights_path)
        results = model.val(data='dataset.yaml')
        
        results_table.append({
            'Model': name,
            'mAP@0.5': round(results.box.map50, 4),
            'mAP@0.5:0.95': round(results.box.map, 4),
            'Precision': round(results.box.mp, 4),
            'Recall': round(results.box.mr, 4),
        })
    else:
        print(f"[SKIP] {name}: weights not found at {weights_path}")

df = pd.DataFrame(results_table)
print("\n" + "=" * 60)
print("DETECTION RESULTS: Baseline vs Outdoor Augmented")
print("=" * 60)
print(df.to_string(index=False))
df.to_csv(f'{RESULTS_DIR}/detection_comparison.csv', index=False)

In [ ]:
import pandas as pd

# Validate each model
models = {
    'YOLOv8n_raw': f'{RESULTS_DIR}/yolov8n_raw/weights/best.pt',
    'YOLOv8n_enhanced': f'{RESULTS_DIR}/yolov8n_enhanced/weights/best.pt',
    'RT-DETR_raw': f'{RESULTS_DIR}/rtdetr_raw/weights/best.pt',
}

results_table = []

for name, weights_path in models.items():
    if os.path.exists(weights_path):
        print(f"\nValidating: {name}")
        model = YOLO(weights_path)
        results = model.val(data='dataset.yaml')
        
        results_table.append({
            'Model': name,
            'mAP@0.5': round(results.box.map50, 4),
            'mAP@0.5:0.95': round(results.box.map, 4),
            'Precision': round(results.box.mp, 4),
            'Recall': round(results.box.mr, 4),
        })
    else:
        print(f"[SKIP] {name}: weights not found at {weights_path}")

df = pd.DataFrame(results_table)
print("\n" + "=" * 60)
print("DETECTION RESULTS COMPARISON")
print("=" * 60)
print(df.to_string(index=False))
df.to_csv(f'{RESULTS_DIR}/detection_comparison.csv', index=False)

import matplotlib.pyplot as plt
import cv2

# Run inference on sample images using best outdoor-augmented model
best_model_path = f'{RESULTS_DIR}/yolov8n_outdoor_aug/weights/best.pt'
if not os.path.exists(best_model_path):
    best_model_path = f'{RESULTS_DIR}/yolov8n_baseline/weights/best.pt'

if os.path.exists(best_model_path):
    model = YOLO(best_model_path)
    
    # Get sample test images
    import glob
    test_imgs = glob.glob(f'{DATASETS_DIR}/*_processed/test/*.jpg')[:6]
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    for idx, img_path in enumerate(test_imgs):
        results = model(img_path, verbose=False)
        annotated = results[0].plot()
        
        ax = axes[idx // 3][idx % 3]
        ax.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
        ax.axis('off')
    
    plt.suptitle('YOLOv8 Detection Results (Outdoor Augmented)', fontsize=16)
    plt.tight_layout()
    plt.savefig(f'{RESULTS_DIR}/detection_samples.png', dpi=150, bbox_inches='tight')
    plt.show()
else:
    print("Train the models first to see results.")

In [ ]:
import matplotlib.pyplot as plt

# Run inference on sample images
best_model_path = f'{RESULTS_DIR}/yolov8n_raw/weights/best.pt'
if os.path.exists(best_model_path):
    model = YOLO(best_model_path)
    
    # Get sample test images
    import glob
    test_imgs = glob.glob(f'{DATASETS_DIR}/*_processed/test/*.jpg')[:6]
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    for idx, img_path in enumerate(test_imgs):
        results = model(img_path, verbose=False)
        annotated = results[0].plot()
        
        ax = axes[idx // 3][idx % 3]
        ax.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
        ax.axis('off')
    
    plt.suptitle('YOLOv8 Detection Results', fontsize=16)
    plt.tight_layout()
    plt.savefig(f'{RESULTS_DIR}/detection_samples.png', dpi=150, bbox_inches='tight')
    plt.show()
else:
    print("Train the models first to see results.")

In [ ]:
print(f"\nPhase 3 results saved to: {RESULTS_DIR}")
print("Next: Open Phase4_Face_Recognition.ipynb")